In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
train_path = r'D:\Machine Learning\Tensorflow\Data\train'
val_path = r'D:\Machine Learning\Tensorflow\Data\validation'
test_path = r'D:\Machine Learning\Tensorflow\Data\test'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
            rescale = 1/255,
            rotation_range = 40,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            #brightness_range = (0.05,0.2),
            shear_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True
            )

val_datagen = ImageDataGenerator(rescale = 1/255)

In [4]:
train_gen = train_datagen.flow_from_directory(
            train_path,
            batch_size = 20,
            target_size = (150,150),
            class_mode = 'binary',
            #save_to_dir = r'D:\Machine Learning\Tensorflow\Data\newTrain'
            )

val_gen = val_datagen.flow_from_directory(
            val_path,
            batch_size = 20,
            target_size = (150,150),
            class_mode = 'binary',
            )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
class Callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs['accuracy']>0.97 and logs['val_accuarcy']>0.9):
            print("\n Training ends because 97% accuracy and 90% validation accuracy is achieved")
            self.model.stop_training = True
            
callback = Callback()
    

In [6]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32 , (3,3) , activation = 'relu', input_shape = (150,150,3)),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(64 , (3,3) , activation = 'relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(128 , (3,3) , activation = 'relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(128 , (3,3) , activation = 'relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])

model.compile(loss = 'binary_crossentropy' , optimizer = tf.keras.optimizers.RMSprop(lr=1e-4) , metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [7]:
model.fit(
        train_gen,
        steps_per_epoch = 100,
        epochs = 100,
        validation_data = val_gen,
        validation_steps = 50,
        verbose = 1,
        callbacks = [callback]
        )

Epoch 1/100
100/100 [==============================] - 76s 756ms/step - loss: 0.6925 - accuracy: 0.5185 - val_loss: 0.6915 - val_accuracy: 0.5150
Epoch 2/100
100/100 [==============================] - 68s 682ms/step - loss: 0.6871 - accuracy: 0.5560 - val_loss: 0.6735 - val_accuracy: 0.5620
Epoch 3/100
100/100 [==============================] - 68s 679ms/step - loss: 0.6778 - accuracy: 0.5790 - val_loss: 0.6566 - val_accuracy: 0.6110
Epoch 4/100
100/100 [==============================] - 74s 741ms/step - loss: 0.6635 - accuracy: 0.6090 - val_loss: 0.7450 - val_accuracy: 0.5290
Epoch 5/100
100/100 [==============================] - 65s 652ms/step - loss: 0.6410 - accuracy: 0.6240 - val_loss: 0.6241 - val_accuracy: 0.6280
Epoch 6/100
100/100 [==============================] - 65s 651ms/step - loss: 0.6306 - accuracy: 0.6330 - val_loss: 0.6111 - val_accuracy: 0.6590
Epoch 7/100
100/100 [==============================] - 65s 651ms/step - loss: 0.6080 - accuracy: 0.6605 - val_loss: 0.5914 -

Epoch 57/100
100/100 [==============================] - 60s 601ms/step - loss: 0.4563 - accuracy: 0.7775 - val_loss: 0.4933 - val_accuracy: 0.7650
Epoch 58/100
100/100 [==============================] - 59s 594ms/step - loss: 0.4624 - accuracy: 0.7760 - val_loss: 0.4301 - val_accuracy: 0.8080
Epoch 59/100
100/100 [==============================] - 59s 591ms/step - loss: 0.4361 - accuracy: 0.7950 - val_loss: 0.4693 - val_accuracy: 0.7730
Epoch 60/100
100/100 [==============================] - 58s 583ms/step - loss: 0.4485 - accuracy: 0.7910 - val_loss: 0.4792 - val_accuracy: 0.7750
Epoch 61/100
100/100 [==============================] - 59s 594ms/step - loss: 0.4414 - accuracy: 0.8020 - val_loss: 0.4329 - val_accuracy: 0.7970
Epoch 62/100
100/100 [==============================] - 58s 585ms/step - loss: 0.4527 - accuracy: 0.7885 - val_loss: 0.4344 - val_accuracy: 0.8070
Epoch 63/100
100/100 [==============================] - 58s 585ms/step - loss: 0.4374 - accuracy: 0.8060 - val_loss: 0

## Here it got 80.5% Accuracy on Test Data

In [40]:
from keras.models import model_from_json

# convert  model to JSON
model_json = model.to_json()
with open(r"D:\Machine Learning\Tensorflow\Models\modelDog_Cat.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("D:\Machine Learning\Tensorflow\Models\modelDog_Cat.h5")
print("Saved model to disk")

Saved model to disk


In [37]:
model2 = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32 , (3,3) , activation = 'relu', input_shape = (150,150,3)),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(32 , (3,3) , activation = tf.nn.leaky_relu),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(128 , (3,3) , activation = tf.nn.leaky_relu),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(256 , (3,3) , activation = tf.nn.leaky_relu),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])

#lr_change = tf.keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate=0.1,
#    decay_steps=10000,
#    decay_rate=0.9)

model2.compile(loss = 'binary_crossentropy' , 
              optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9) ,
              metrics = ['accuracy'])
model2.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 34, 34, 128)       36992     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 15, 15, 256)     

In [38]:
model2.fit(
        train_gen,
        steps_per_epoch = 100,
        epochs = 100,
        validation_data = val_gen,
        validation_steps = 50,
        verbose = 1,
        callbacks = [callback]
        )

Epoch 1/100
100/100 [==============================] - 56s 561ms/step - loss: 0.6939 - accuracy: 0.4995 - val_loss: 0.6863 - val_accuracy: 0.5140
Epoch 2/100
100/100 [==============================] - 54s 539ms/step - loss: 0.6901 - accuracy: 0.5345 - val_loss: 0.6853 - val_accuracy: 0.5830
Epoch 3/100
100/100 [==============================] - 54s 542ms/step - loss: 0.6897 - accuracy: 0.5290 - val_loss: 0.6765 - val_accuracy: 0.6380
Epoch 4/100
100/100 [==============================] - 55s 548ms/step - loss: 0.6838 - accuracy: 0.5555 - val_loss: 0.6699 - val_accuracy: 0.5550
Epoch 5/100
100/100 [==============================] - 53s 533ms/step - loss: 0.6869 - accuracy: 0.5465 - val_loss: 0.6699 - val_accuracy: 0.6130
Epoch 6/100
100/100 [==============================] - 55s 553ms/step - loss: 0.6770 - accuracy: 0.5825 - val_loss: 0.6838 - val_accuracy: 0.5200
Epoch 7/100
100/100 [==============================] - 53s 526ms/step - loss: 0.6661 - accuracy: 0.5985 - val_loss: 0.6400 -

Epoch 57/100
100/100 [==============================] - 53s 527ms/step - loss: 0.4612 - accuracy: 0.7845 - val_loss: 0.4367 - val_accuracy: 0.7960
Epoch 58/100
100/100 [==============================] - 53s 530ms/step - loss: 0.4783 - accuracy: 0.7745 - val_loss: 0.4710 - val_accuracy: 0.7750
Epoch 59/100
100/100 [==============================] - 53s 525ms/step - loss: 0.4559 - accuracy: 0.7770 - val_loss: 0.4242 - val_accuracy: 0.8120
Epoch 60/100
100/100 [==============================] - 53s 528ms/step - loss: 0.4503 - accuracy: 0.7825 - val_loss: 0.4305 - val_accuracy: 0.8000
Epoch 61/100
100/100 [==============================] - 53s 526ms/step - loss: 0.4645 - accuracy: 0.7740 - val_loss: 0.4745 - val_accuracy: 0.7740
Epoch 62/100
100/100 [==============================] - 53s 525ms/step - loss: 0.4614 - accuracy: 0.7815 - val_loss: 0.4434 - val_accuracy: 0.7890
Epoch 63/100
100/100 [==============================] - 52s 525ms/step - loss: 0.4644 - accuracy: 0.7865 - val_loss: 0

## Here it got 82% Accuracy on Test Data

In [41]:
# convert  model to JSON
model_json2 = model2.to_json()
with open(r"D:\Machine Learning\Tensorflow\Models\modelDog_Cat2.json", "w") as json_file:
    json_file.write(model_json2)
# serialize weights to HDF5
model2.save_weights("D:\Machine Learning\Tensorflow\Models\modelDog_Cat2.h5")
print("Saved model to disk")

Saved model to disk
